In [1]:
import os
import jax
import flax

In [2]:
import xpag
from xpag.wrappers import gym_vec_env
from xpag.buffers import DefaultEpisodicBuffer
from xpag.samplers import DefaultEpisodicSampler, HER
from xpag.goalsetters import DefaultGoalSetter
from xpag.agents import SAC
from xpag.tools import learn
from xpag.tools import mujoco_notebook_replay

In [3]:
from xpag.envs import *

REGISTERING DubinsMazeEnv
REGISTERING DubinsMazeEnv


In [4]:
import copy
import numpy as np

In [15]:
num_envs = 1  # the number of rollouts in parallel during training
env, eval_env, env_info = gym_vec_env('DubinsMazeEnvGCPHERSB3-v0', num_envs)

TypeError: __init__() missing 5 required positional arguments: 'L_full_demonstration', 'L_states', 'L_inner_states', 'L_goals', and 'L_budgets'

In [5]:
env = gym.make('DubinsMazeEnv-v0')

MAZE setting random seed  0


/Users/chenu/opt/anaconda3/envs/xpag_env/lib/python3.9/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [6]:
env.max_episode_steps

30

In [7]:
env.spec

EnvSpec(entry_point='xpag.envs.dubins_mazeenv.mazeenv_cst_speed:DubinsMazeEnv', reward_threshold=None, nondeterministic=False, max_episode_steps=30, order_enforce=True, kwargs={}, namespace=None, name='DubinsMazeEnv', version=0)

In [8]:
env_hc = gym.make('HalfCheetah-v3')

objc[30605]: Class GLFWApplicationDelegate is implemented in both /Users/chenu/.mujoco/mujoco200/bin/libglfw.3.dylib (0x15088d778) and /usr/local/Cellar/glfw/3.3.2/lib/libglfw.3.3.dylib (0x15092cf10). One of the two will be used. Which one is undefined.
objc[30605]: Class GLFWWindowDelegate is implemented in both /Users/chenu/.mujoco/mujoco200/bin/libglfw.3.dylib (0x15088d700) and /usr/local/Cellar/glfw/3.3.2/lib/libglfw.3.3.dylib (0x15092cf38). One of the two will be used. Which one is undefined.
objc[30605]: Class GLFWContentView is implemented in both /Users/chenu/.mujoco/mujoco200/bin/libglfw.3.dylib (0x15088d7a0) and /usr/local/Cellar/glfw/3.3.2/lib/libglfw.3.3.dylib (0x15092cf88). One of the two will be used. Which one is undefined.
objc[30605]: Class GLFWWindow is implemented in both /Users/chenu/.mujoco/mujoco200/bin/libglfw.3.dylib (0x15088d818) and /usr/local/Cellar/glfw/3.3.2/lib/libglfw.3.3.dylib (0x15092d000). One of the two will be used. Which one is undefined.


In [9]:
env_hc.spec

EnvSpec(entry_point='gym.envs.mujoco.half_cheetah_v3:HalfCheetahEnv', reward_threshold=4800.0, nondeterministic=False, max_episode_steps=1000, order_enforce=True, kwargs={}, namespace=None, name='HalfCheetah', version=3)

In [8]:
num_envs = 1  # the number of rollouts in parallel during training
env, eval_env, env_info = gym_vec_env('DubinsMazeEnv-v0', num_envs)

MAZE setting random seed  0
MAZE setting random seed  0


Process Worker<AsyncVectorEnv>-0:
Traceback (most recent call last):
  File "/Users/chenu/opt/anaconda3/envs/xpag_env/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/chenu/opt/anaconda3/envs/xpag_env/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/chenu/Desktop/PhD/github/xpag/xpag/wrappers/gym_vec_env.py", line 152, in _worker_shared_memory_no_auto_reset
    env = env_fn()
  File "/Users/chenu/opt/anaconda3/envs/xpag_env/lib/python3.9/site-packages/gym/vector/utils/misc.py", line 22, in __call__
    return self.fn()
  File "/Users/chenu/Desktop/PhD/github/xpag/xpag/wrappers/gym_vec_env.py", line 75, in <lambda>
    [lambda: ResetDoneWrapper(gym.make(env_name))] * num_envs,
  File "/Users/chenu/opt/anaconda3/envs/xpag_env/lib/python3.9/site-packages/gym/envs/registration.py", line 676, in make
    return registry.make(id, **kwargs)
  File "/Users/chenu/opt/anaconda3

EOFError: 

In [10]:
env

<ResetDoneVecWrapper<AsyncVectorEnv instance>>

In [20]:
env_info = {}
env_info['observation_dim'] = 3
env_info['desired_goal_dim'] = 2
env_info['action_dim'] = 1
env_info['max_episode_steps'] = 20
env_info['is_goalenv'] = True

In [21]:
agent = SAC(
    env_info['observation_dim'] if not env_info['is_goalenv']
    else env_info['observation_dim'] + env_info['desired_goal_dim'],
    env_info['action_dim'],
    {}
)

In [22]:
sampler =  HER(test_env.compute_reward)

In [23]:
buffer = DefaultEpisodicBuffer(
    max_episode_steps=env_info['max_episode_steps'],
    buffer_size=1_000_000,
    sampler=sampler
)

In [12]:
L_states = [np.array([0.5, 0.5, 0.]), np.array([1.8, 1., np.pi/2]), np.array([0.5, 1.8, np.pi])]
L_full_demonstration = [np.array([0.5, 0.5, 0.]), np.array([1.8, 1., np.pi/2]), np.array([0.5, 1.8, np.pi])]
L_full_inner_demonstration = copy.deepcopy(L_states)
L_goals = [state[:2] for state in L_states]
L_inner_states = copy.deepcopy(L_states)
L_budgets = [15, 15]

In [13]:
test_env = gym.make('DubinsMazeEnvGCPHERSB3-v0', L_full_demonstration = L_full_demonstration,
                                  L_states =  L_states,
                                  L_goals =  L_goals,
                                  L_inner_states =  L_inner_states,
                                  L_budgets =  L_budgets)

MAZE setting random seed  0


In [ ]:
test_env.reset()